# Coefficient Reloadable FIR Filter Hardware Design
This notebook demonstrates how to accelerate a function (FIR filter) from a processing system (PS) to programmable logic (PL), and how to reload FIR filter coefficients on a Zynq-7000 using PYNQ. First, input the voice signal. Then, realize the noise reduction of the voice signal in the PL, and then play the voice after noise reduction

## 查看音频参数

In [1]:
from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/entire_FIR/test_recording.wav")  #recording_0  test_recording

In [32]:
%matplotlib inline
import wave
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft

wav_path = "/home/xilinx/jupyter_notebooks/entire_FIR/test_recording.wav"

with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()



temp_buffer = np.empty((num_frames, num_channels, 4), dtype = np.uint8) #int8
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, sample_width)
temp_buffer[:, :, sample_width:] = (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 1023
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])
print(frames,temp_buffer)
print(num_frames,num_channels,sample_rate,sample_width,raw_bytes)    

[[  -71]
 [  742]
 [ 1490]
 ..., 
 [-2096]
 [-1437]
 [ -691]] [[[185 255 255 255]]

 [[230   2   0   0]]

 [[210   5   0   0]]

 ..., 
 [[208 247 255 255]]

 [[ 99 250 255 255]]

 [[ 77 253 255 255]]]
240000 1 48000 2 [185 255 230 ..., 250  77 253]


# Program FPGA

In [33]:
from pynq import Overlay

# Program PL + load drivers for hardware found in HWH
overlay = Overlay("/home/xilinx/jupyter_notebooks/entire_FIR/fir_reload.bit")

## Load Drivers

In [34]:
from pynq.lib import AxiGPIO
from pynq import allocate

# Load FIR Data DMA
dma_fir_data = overlay.axi_dma_fir_data
# Load FIR Reload DMA
dma_fir_reload = overlay.axi_dma_reload
# Load FIR Config DMA
dma_fir_config = overlay.axi_dma_config

## paramater value

In [35]:
import numpy as np
import time
n=num_frames
T = n/num_frames
t = np.linspace(0,T,n,endpoint=False)
n_plot = 100

## Send Waveform to FIR Filter via DMA with initial coefficients

In [44]:
# Allocate physically contiguous buffers for direct register DMA
in_data_buffer = allocate(shape=(n,2),dtype=np.int32) #int32
print(in_data_buffer[:,:])
out_data_buffer = allocate(shape=(n,2),dtype=np.int32)

# Copy the samples to the input buffer
#for channel_index in range(num_channels):
#    samples = frames[:, channel_index]
#    samples = samples.astype(np.int32)
#    print(samples)
#    np.copyto(in_data_buffer[:, channel_index],samples)
samples =  frames   
samples = samples.astype(np.int32)
np.copyto(in_data_buffer,samples)


#samples = samples.astype(np.int32)
#np.copyto(in_data_buffer,samples)
# Trigger the DMA and wait for the result
start_time = time.time()
dma_fir_data.sendchannel.transfer(in_data_buffer)
dma_fir_data.recvchannel.transfer(out_data_buffer)
dma_fir_data.sendchannel.wait()
dma_fir_data.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
print('Hardware FIR execution time: ',hw_exec_time,'s for ',T,'s signal')
#print('Hardware acceleration factor: ',sw_exec_time_lpf / hw_exec_time)
print("滤波前的音频数据")
print(in_data_buffer[:,:])
print("滤波后的音频数据")
print(out_data_buffer[:,:])
out_data_buffer[:,:]=out_data_buffer[:,:]*30
out_data_buffer.close()



[[0 0]
 [0 0]
 [0 0]
 ..., 
 [0 0]
 [0 0]
 [0 0]]
Hardware FIR execution time:  0.007203578948974609 s for  1.0 s signal
滤波前的音频数据
[[  -71   -71]
 [  742   742]
 [ 1490  1490]
 ..., 
 [-2096 -2096]
 [-1437 -1437]
 [ -691  -691]]
滤波后的音频数据
[[-14 -19]
 [-16 -20]
 [-19 -20]
 ..., 
 [  0 -10]
 [ -5 -14]
 [-10 -17]]


In [45]:
from scipy.io.wavfile import write
write("还原_test.wav", sample_rate, out_data_buffer.astype(np.int16))

from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/entire_FIR/还原_test.wav")